In [8]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd

In [9]:
ee.Initialize()

# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')

# AREA OF INTEREST
region_L1 = ee.Geometry.Polygon([[[-30, -40], [65, -30], [65, 40], [-30, 40]]])
region_json= [[-30, -40], [65, -30], [65, 40], [-30, 40]]

# <span style="color:red">INPUTS</span> 

In [43]:
subregions = {
  'na': 'Northern Africa',
  'ma': 'Middle Africa',
  'wa': 'Western Africa', 
  'ea': 'Eastern Africa',
  'sa': 'Southern Africa', 
  'ws': 'Western Asia'
}

year = 2016
name_asset = 'T_Annual'

In [44]:
radice = 'projects/fao-wapor/'
raster_name = radice + name_asset + "/" + name_asset + "-" + str(year)
print(raster_name)
raster_stats = ee.Image(raster_name).toFloat()
scale_calc = ee.Image(raster_stats).projection().nominalScale().getInfo()
print('Image scale: ', scale_calc)    
for code,subregion in subregions.iteritems():
    #print(code)
    selected_subregion = subregions[code]
    #print selected_subregion
    filtered = countries_gaul.filter(ee.Filter.eq('subregion', selected_subregion))
    country_number = filtered.size().getInfo()
    #print(country_number)    
    prefix = raster_name.split("/")[2] 
    means = raster_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = scale_calc,
    )

    reducers = ee.Reducer.minMax().combine(
        reducer2 = ee.Reducer.sum(),
        sharedInputs = True
    )

    minMaxStds = raster_stats.reduceRegions(
                collection = filtered,
                reducer = reducers,
                scale = scale_calc,
    )
    features_m = means.getInfo()['features']
    features_mms = minMaxStds.getInfo()['features']
    df_m = pd.DataFrame(data=features_m[0:], columns = features_m[0])
    df_mms = pd.DataFrame(data=features_mms[0:], columns = features_mms[0])
    serie_m = df_m['properties'].apply(pd.Series)
    serie_mms = df_mms['properties'].apply(pd.Series)
    df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])
    df_mms = pd.DataFrame(serie_mms[['fid', 'min', 'max','sum']])
    df_stats = df_m.set_index('fid').join(df_mms.set_index('fid'), lsuffix='_df_m', rsuffix='_df_mms')
    file_name = name_asset + "_" + str(year) + "_" + code 
    print(file_name)
    df_stats.to_csv('stats/' + file_name + '.csv')

projects/fao-wapor/T_Annual/T_Annual-2016
('Image scale: ', 248.48100621908284)
T_Annual_2016_ma
T_Annual_2016_wa
T_Annual_2016_na
T_Annual_2016_ea
T_Annual_2016_ws
T_Annual_2016_sa
